In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader,TensorDataset
from torchvision import models

In [ ]:
train_loader = DataLoader(dataset=TensorDataset(train_data,train_labels),
                         batch_size=batch_size,
                         shuffle=True) #还可以加num_workers=4,dataloader一次性创建num_worker个工作进程，加速
test_loader = DataLoader(dataset=TensorDataset(test_data,test_labels),
                        batch_size=batch_size,
                        shuffle=False)

In [ ]:
epochs = 25
BestEpoch = 0
BestSim = 0

model = models.resnet34(pretrained=True)
model = torch.compile(model) #模型加速


In [ ]:
criterion = nn.NLLLoss()
optimizer = torch.optim.AdamW(model.parameters(),lr=1e-4)
'''
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0001, weight_decay=1e-5)

for循环以上的参数固定, 只训练下面的参数
for p in moddel.parameters():
    p.requires_grad = False
'''

#### 训练循环

In [ ]:
#model.train()的作用是启用 Batch Normalization 和 Dropout
#model.eval()的作用是不启用 Batch Normalization 和 Dropout
for epoch in range(epochs):
    epoch_loss = 0
    for i,batch_data in enumerate(tqdm(train_loader)):
        batch_inputs,batch_labels = batch_data
        batch_inputs,batch_labels = batch_inputs.to(device),batch_labels.to(device)
        
        pred = model(batch_inputs)
        
        loss = criterion(pred,batch_lables)
        
        optimizer.zero_grad()
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    epoch_loss /= len(train_loader)
    print(f"epoch: {epoch}, training loss: {epoch_loss}")
    
    '''test loss'''
    epoch_loss = 0
    with torch.no_grad():
        for batch_inputs,batch_labels in tqdm(test_loader):
            batch_inputs,batch_labels = batch_inputs.to(device),batch_labels.to(device)
            pred = model(batch_inputs)
            loss = criterion(pred,batch_labels)
            epoch_loss += loss.item()
        epoch_loss /= len(train_loader)
        print(f"epoch: {epoch}, test loss: {epoch_loss}")
        
    if epoch_loss > BestSim:
        BestSim = epoch_loss
        BestEpoch = epoch + 1
        print(f"save best model at {BestSim} with epoch {BestEpoch}")
        torch.save(model.state_dict(),'checkpoint_model.pt')
        
    if epoch - 3 > BestEpoch:
        print(f"Early stop at {epoch+1} with best epoch {BestEpoch} and test similarity {BestSim}.")
        break
        
        